# Create Heatmaps for E2F Transcription

This notebook looks at the significant genes in at least one cancer. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [ ]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [4]:
prot_enr.res2d.iloc[[15]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
15,E2F transcription factor network Homo sapiens ...,19/72,0.001787,0.023337,0,0,2.021363,12.79011,PRMT5;RRM1;CDKN1B;CBX5;APAF1;HDAC1;XRCC1;YY1;D...,NCI-Nature_2016


# Step 2: Get the list of significant genes from certain pathways

In [6]:
i = 15
trans = prot_enr.res2d.Genes[i]
trans_genes = trans.split(';')
print(prot_enr.res2d.Term[i])
print('total genes:',len(trans_genes))

E2F transcription factor network Homo sapiens bb4d0fd3-6191-11e5-8ac5-06603eb7f303
total genes: 19


# Step 3: Create HeatMap

Slice out the significant genes in the pathway from sig_df (sig_pval_heatmap.csv).

In [26]:
bool_df = sig_df.Proteomics.isin(trans_genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

19

In [27]:
# Only include p-values < 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= 0.05]

In [28]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height=400, plot_width=600)

Loading BokehJS ...

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [11]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [12]:
get = pos_neg_df.Proteomics.isin(trans_genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM3', 'RRM1', 'TOPBP1', 'TK1', 'HDAC1', 'YY1', 'E2F5', 'SP1'],
      dtype=object)

In [13]:
# Only include p-values < 0.01
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= 0.01]

In [14]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 400, plot_width=500)

Loading BokehJS ...

In [11]:
# all mcm and rfc 

In [15]:
df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(df.Proteomics.unique()) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')


In [20]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Ribosome biogenesis in eukaryotes Homo sapiens...,13/89,1.180367e-09,3.458475e-07,0,0,9.393403,193.104322,UTP6;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;PWP2;UT...,KEGG_2016
1,DNA replication Homo sapiens hsa03030,9/36,3.083251e-09,4.516962e-07,0,0,16.077170,315.068832,RFC3;PCNA;RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM2,KEGG_2016
2,B cell receptor signaling pathway Homo sapiens...,11/73,1.626924e-08,1.588963e-06,0,0,9.690349,173.786622,PPP3CB;SYK;CHUK;PIK3CA;PRKCB;INPP5D;BLNK;RAC2;...,KEGG_2016
3,Fc gamma R-mediated phagocytosis Homo sapiens ...,12/93,2.188722e-08,1.603239e-06,0,0,8.297894,146.352976,PAK1;PTPRC;SYK;PIK3CA;MYO10;PRKCB;INPP5D;RAC2;...,KEGG_2016
4,RNA transport Homo sapiens hsa03013,15/172,8.324097e-08,4.877921e-06,0,0,5.608315,91.424098,RANBP2;NUP210;NUP155;NCBP1;NUP133;THOC3;THOC2;...,KEGG_2016


In [21]:
genes = prot_enr.res2d.Genes[1]
genes = genes.split(';')
genes

['RFC3', 'PCNA', 'RFC4', 'MCM7', 'RFC2', 'MCM3', 'MCM4', 'MCM5', 'MCM2']

In [22]:
get = df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_k = df[get] # Keep only genes with pos and neg
genes_k.Proteomics.unique()

array(['MCM4', 'PCNA', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC3', 'RFC2',
       'RFC4'], dtype=object)

In [23]:
# Only include p-values < 0.01
genes_k = genes_k.loc[genes_k['P_Value'] <= 0.01]

In [24]:
pu.plotCircleHeatMap(genes_k, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 400, plot_width=800)

Loading BokehJS ...

In [25]:
# Look at p-values 
certain_gene = 'E2F5'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
1841,E2F5,0.007906,0.413613,Gbm
49317,E2F5,0.371901,-0.457100,Brca
61570,E2F5,0.641133,0.039176,Ov
67858,E2F5,0.078407,-0.282000,En
